# Requirements

In [1]:
import tenseal as ts # pip install tenseal
from deepface import DeepFace #!pip install deepface
import base64
from deepface.commons import distance as dst

# Finding embeddings

We are going to find vector representations of facial images. This will be done in the client side.

In [2]:
img1_path = "deepface/tests/dataset/img1.jpg"
img2_path = "deepface/tests/dataset/img2.jpg"

In [3]:
img1_embedding = DeepFace.represent(img1_path, model_name = 'Facenet')
img2_embedding = DeepFace.represent(img2_path, model_name = 'Facenet')

# Commons

In [4]:
def write_data(file_name, data):
    
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)
        
    with open(file_name, 'wb') as f: 
        f.write(data)

def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    
    #base64 to bytes
    return base64.b64decode(data)

# Initialization

We are going to generate secret - public key pair in this stage. This will be done in the client side.

In [5]:
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree = 8192,
            coeff_mod_bit_sizes = [60, 40, 40, 60]
          )

context.generate_galois_keys()
context.global_scale = 2**40

In [6]:
secret_context = context.serialize(save_secret_key = True)
write_data("secret.txt", secret_context)

In [7]:
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()
write_data("public.txt", public_context)

In [8]:
del context, secret_context, public_context

# Encryption

We are going to apply homomorphic encryption to facial embeddings. This will be done in the client side. 

Then, homomorphic encrypted facial embeddings will be stored in the cloud.

In [9]:
context = ts.context_from(read_data("secret.txt"))

In [10]:
enc_v1 = ts.ckks_vector(context, img1_embedding)
enc_v2 = ts.ckks_vector(context, img2_embedding)

In [11]:
enc_v1_proto = enc_v1.serialize()
enc_v2_proto = enc_v2.serialize()

In [12]:
write_data("enc_v1.txt", enc_v1_proto)
write_data("enc_v2.txt", enc_v2_proto)

In [13]:
del context, enc_v1, enc_v2, enc_v1_proto, enc_v2_proto

# Calculations

Once homomorphic encrypted facial embeddings stored in the cloud, we are able to make calculations on encrypted data.

Notice that we just have public key here and we don't have secret key.

In [14]:
context = ts.context_from(read_data("public.txt"))

In [15]:
enc_v1_proto = read_data("enc_v1.txt")
enc_v2_proto = read_data("enc_v2.txt")

In [16]:
enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
enc_v1.link_context(context)

enc_v2 = ts.lazy_ckks_vector_from(enc_v2_proto)
enc_v2.link_context(context)

In [17]:
euclidean_squared = enc_v1 - enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)

In [18]:
write_data("euclidean_squared.txt", euclidean_squared.serialize())

In [19]:
#we must not decrypt the homomorphic encrypted euclidean squared value in this stage
#because we don't have the secret key. check this operation. it should throw an exception!

try:
    euclidean_squared.decrypt()
except Exception as err:
    print("Exception: ", str(err))

Exception:  the current context of the tensor doesn't hold a secret_key, please provide one as argument


In [20]:
del context, enc_v1_proto, enc_v2_proto, enc_v1, enc_v2, euclidean_squared

# Decryption

Once homomorphic encrypted euclidean squared value found in the cloud, we are going to retrieve it to the client side.

Client can decrypt it because we have the secret key in the client side.

In [21]:
context = ts.context_from(read_data("secret.txt"))

In [22]:
euclidean_squared_proto = read_data("euclidean_squared.txt")

In [23]:
euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
euclidean_squared.link_context(context)

In [24]:
euclidean_squared_plain = euclidean_squared.decrypt()[0]

In [25]:
euclidean_squared_plain

66.36774485705642

In [26]:
if euclidean_squared_plain < 100:
    print("they are same person")
else:
    print("they are different persons")

they are same person


# Validation

What if euclidean distance calculation is done in the client side always? Result should be same!

In [27]:
distance = dst.findEuclideanDistance(img1_embedding, img2_embedding)

In [28]:
print("euclidean squared - tradational: ", distance * distance)
print("euclidean squared - homomorphic: ", euclidean_squared_plain)

euclidean squared - tradational:  66.3677359167053
euclidean squared - homomorphic:  66.36774485705642


In [29]:
#check the difference is acceptable
abs(distance * distance - euclidean_squared_plain) < 0.00001

True